In [1]:
## pip install pillow
## pip install torch
## pip install torchvision

In [2]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [ ]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

In [5]:
image, label = train_data[0]

In [6]:
image.size()

torch.Size([3, 32, 32])

In [7]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [8]:
class NeuralNet(nn.Module):

  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(3, 12, 5)  # (12, 28, 28)
    self.pool = nn.MaxPool2d(2, 2)  # (12, 14, 14)
    self.conv2 = nn.Conv2d(12, 24, 5)  # (24, 10, 10) --> (24, 5, 5)
    self.fc1 = nn.Linear(24 * 5 * 5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)  # Flatten starting from dimension 1 (batch dimension is 0)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [9]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [10]:
for epoch in range(30):
  print(f"Training epoch {epoch + 1} ...")

  running_loss = 0.0
  for i, data in enumerate(train_loader):
    inputs, labels = data

    optimizer.zero_grad()

    outputs = net(inputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f"Loss: {running_loss / len(train_loader):.4f}")

Training epoch 1 ...
Loss: 2.1132
Training epoch 2 ...
Loss: 1.6898
Training epoch 3 ...
Loss: 1.5104
Training epoch 4 ...
Loss: 1.3930
Training epoch 5 ...
Loss: 1.2982
Training epoch 6 ...
Loss: 1.2139
Training epoch 7 ...
Loss: 1.1467
Training epoch 8 ...
Loss: 1.0845
Training epoch 9 ...
Loss: 1.0375
Training epoch 10 ...
Loss: 0.9953
Training epoch 11 ...
Loss: 0.9514
Training epoch 12 ...
Loss: 0.9181
Training epoch 13 ...
Loss: 0.8820
Training epoch 14 ...
Loss: 0.8502
Training epoch 15 ...
Loss: 0.8198
Training epoch 16 ...
Loss: 0.7953
Training epoch 17 ...
Loss: 0.7645
Training epoch 18 ...
Loss: 0.7390
Training epoch 19 ...
Loss: 0.7108
Training epoch 20 ...
Loss: 0.6873
Training epoch 21 ...
Loss: 0.6628
Training epoch 22 ...
Loss: 0.6428
Training epoch 23 ...
Loss: 0.6176
Training epoch 24 ...
Loss: 0.5922
Training epoch 25 ...
Loss: 0.5704
Training epoch 26 ...
Loss: 0.5517
Training epoch 27 ...
Loss: 0.5297
Training epoch 28 ...
Loss: 0.5090
Training epoch 29 ...
Loss: 0

In [11]:
torch.save(net.state_dict(), 'trained_net.pth')

In [ ]:
net.load_state_dict(torch.load('trained_net.pth'))

In [13]:
correct = 0
total = 0

net.eval()


with torch.no_grad():
  for data in test_loader:
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()


accuracy = 100 * correct / total

print(f"Accuracy: {accuracy}%")

Accuracy: 67.78%


In [15]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


def load_image(image_path):
  image = Image.open(image_path)
  image = new_transform(image)
  image = image.unsqueeze(0)
  return image


image_paths = ['image1.jpg', 'image2.jpg']
images = [load_image(img) for img in image_paths]


net.eval()
with torch.no_grad():
  for image in images:
    output = net(image)
    _, predicted = torch.max(output, 1)
    print(f"Prediction: {class_names[predicted.item()]}")

Prediction: dog
Prediction: plane
